# Preprocessing and Feature Selection for RSF model for Living Donors

## Importing packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sksurv.datasets import load_gbsg2
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.metrics import concordance_index_ipcw
from sksurv.linear_model import CoxPHSurvivalAnalysis

set_config(display="text") 

In [ ]:
pd.set_option('display.max_columns', 1200)
pd.set_option('display.max_rows', 300)

## Loading data into DataFrames and joining them

In [ ]:
kidpan_df = pd.read_csv('csv_data/Kidney_Pancreas_full.csv')
living_df = pd.read_csv('csv_data/Living_Donor.csv')

In [ ]:
kidpan_living = pd.merge(kidpan_df, living_df, on="DONOR_ID", how="inner")
del kidpan_df
del living_df

## Dropping duplicate columns

In [ ]:
duplicated_columns = []
for col in kidpan_living.columns:
    if col.endswith('_x') and col.rstrip('_x') + '_y' in kidpan_living.columns:
        duplicated_columns.append(col)
duplicated_columns

In [ ]:
different_duplicates = []
for col in duplicated_columns:
    if (kidpan_living[col].equals(kidpan_living[col.rstrip('_x') + '_y'])):
        print(col.rstrip('_x'), "identical")
        kidpan_living.rename(columns={col.rstrip('_x') + '_y': col.rstrip('_x')}, inplace=True)
        kidpan_living.drop(col, axis=1, inplace=True)
    else:
        print(col.rstrip('_x'), "different")
        different_duplicates.append(col.rstrip('_x'))

In [ ]:
kidpan_living.sort_index(axis=1).head(10)

#### Manually handling varying duplicates

In [ ]:
cols_with_identical_DON_y= ['CITIZENSHIP', 'CMV_IGG', 'CMV_IGM', 'EDUCATION', 'GENDER', 'ABO', 'HBV_CORE',
                           'HBV_SUR_ANTIGEN', ]
for col in cols_with_identical_DON_y:
    is_equal = kidpan_living[col + '_DON'].equals(kidpan_living[col+'_y'])
    print(col, is_equal )
    if (is_equal):
        kidpan_living.drop(col+'_y', axis=1, inplace=True)
        kidpan_living.rename(columns={col+'_x': col+'_REC'}, inplace=True)
        different_duplicates.remove(col)


In [ ]:
kidpan_living['ETHCAT_DON_x'] = kidpan_living['ETHCAT_DON_x'].astype(int)
is_equal = kidpan_living['ETHCAT_DON_x'].equals(kidpan_living['ETHCAT_DON_y'])
print('ETHCAT_DON', is_equal)
if(is_equal):
    kidpan_living.drop('ETHCAT_DON_y', axis=1, inplace=True)
    kidpan_living.rename(columns={'ETHCAT_DON_x': 'ETHCAT_DON'}, inplace=True)
    different_duplicates.remove('ETHCAT_DON')
    

In [ ]:
kidpan_living.drop('LIV_DON_TY_y', axis=1, inplace=True)
kidpan_living.rename(columns={'LIV_DON_TY_x': 'LIV_DON_TY'}, inplace=True)
different_duplicates.remove('LIV_DON_TY')

In [ ]:
columns_to_just_rename = ["AGE_BIN", 'REGION']
for col in columns_to_just_rename:
    kidpan_living.rename(columns={col+'_x': col+'_REC',
                                  col+'_y': col+'_DON'}, inplace=True)
    different_duplicates.remove(col)

In [ ]:
print(different_duplicates)
for col in different_duplicates:
    kidpan_living.drop(col+'_x', axis=1, inplace=True)
    kidpan_living.drop(col+'_y', axis=1, inplace=True)
kidpan_living

## Statistics

In [ ]:
print("kidpan_living", kidpan_living.shape)

#### Checking % of NaN values + Keeping track of columns with more than 50% entries missing

In [ ]:
print("How many entries are NaN in kidpan_living")
kidpan_living_columns_to_delete = []
for col in kidpan_living.columns:
    totalRows = kidpan_living.shape[0]
    count = kidpan_living[col].isna().sum()
    percentage = count / totalRows * 100
    if(percentage > 50.00):
        kidpan_living_columns_to_delete.append(col)
    print(f"{col} {percentage:.2f}%")

In [ ]:
print("Number of columns to be dropped from kidpan_living",len(kidpan_living_columns_to_delete))

#### Ensuring we have only living donors and kidney transplantations

In [ ]:
kidpan_living["KIDNEY_RECOV"].value_counts()

In [ ]:
kidpan_living["LIVER_RECOV"].value_counts()

In [ ]:
kidpan_living["DON_ORG"].value_counts()

In [ ]:
kidpan_living["DON_ORG"].isna().sum()

In [ ]:
kidpan_living["DON_TY"].value_counts()

## Dropping unwanted features

#### Dropping columns with with > 50% NaN values

In [ ]:
original_columns = kidpan_living.columns
reduced_columns = [x for x in original_columns if x not in kidpan_living_columns_to_delete]
kidpan_living = kidpan_living[reduced_columns]
kidpan_living

#### What features still have a significant amount of values missing

In [ ]:
NaN_count_df = pd.DataFrame(columns=["Feature", "Percentage of NaN"])
for col in reduced_columns:
    totalRows = kidpan_living.shape[0]
    count = kidpan_living[col].isna().sum()
    percentage = count / totalRows * 100
    new_row = {"Feature": col, "Percentage of NaN": percentage}
    NaN_count_df = pd.concat([NaN_count_df, pd.DataFrame([new_row])])
    
NaN_count_df = NaN_count_df.sort_values(by="Percentage of NaN")
NaN_count_df = NaN_count_df.reset_index(drop=True)
NaN_count_df[0:269]

#### Unwanted features from the Kidney_Pancreas table

In [ ]:
# columns containing data collected post-transplantation
kidpan_post_transplant_columns = ["ACUTE_REJ_EPI_KI", "ACUTE_REJ_EPI_PA", 
                           "ANAST_LK_PA", "BIOP_ISLET_PA", "BLEED_PA",
                          "COMPL_ABSC", "COMPL_ANASLK", "COMPL_PANCREA", "FAILDATE_KI", "FAILDATE_PA",
                          "FIRST_WK_DIAL", "GRF_FAIL_CAUSE_TY_KI", "GRF_FAIL_CAUSE_TY_PA", "GRF_STAT_KI",
                           "GRF_STAT_PA", "GRF_VASC_THROMB_PA", "HBA1C_PA_TRR", "GTIME_PA", "GSTATUS_PA", 
                           "INFECT_PA", "PANCREATIT_PA", "REJ_ACUTE_PA", "REJ_BIOPSY", "REJ_HYPER_PA",
                           "REJCNF_KI", "REJCNF_PA", "REJTRT_KI", "REJTRT_PA", "RESUM_MAINT_DIAL",
                           "RESUM_MAINT_DIAL_DT", "SERUM_CREAT", "COD_KI", "COD_PA", "COD_WL", "COD2_KI",
                          "COD2_PA", "COD3_KI", "COD3_PA", "FUNC_STAT_TRF", "LOS", "PRI_PAYMENT_TRR_KI",
                           "TRTREJ1Y_KI", "TRTREJ6M_KI", "CREAT1Y", "CREAT6M"
                          ]

# columns containing data collected at transplantation
kidpan_at_transplant_columns = ["CADEMIC_LEVEL_TRR", "ACADEMIC_PRG_TRR", "ADMISSION_DATE",
                         "COLD_ISCH_KI", "DISCHARGE_DATE", "DON_TY", "MED_COND_TRR", "FUNC_STAT_TRR",
                        "WORK_INCOME_TRR", "ADMISSION_DATE", "ART_RECON", "CMV_IGG", "CMV_IGM", "CMV_STATUS",
                         "DATA_TRANSPLANT", "DIAL_TRR", "L_FIN_FLOW_RATE_TX", "L_FIN_RESIST_TX",
                         "MED_COND_TRR", "ORG_REC_ON", "PERM_STATE_TRR", "PUMP_KI", "R_FIN_FLOW_RATE_TX",
                         "R_FIN_RESIST_TX", "REC_ON_ICE", "REC_ON_PUMP", "TX_PROCEDUR_TY_KI"
                        ]

# irrelevant columns
kidpan_irrelevant_columns = ["_id", "DONOR_ID", "WL_ID_CODE", "PT_CODE", "STATUS_DDR", "STATUS_LDR", "STATUS_TCR",
                      "STATUS_TRR", "TRR_ID_CODE"
                     ]

# columns regarding pancreas transplantation
kidpan_pancreas_columns = ["ACUTE_REJ_EPI_PA", "AMYLASE", "ANAST_LK_PA", "ART_RECON", "BIOP_ISLET_PA", "BLEED_PA",
                   "BLOOD_SUGAR_DIET_PA", "BLOOD_SUGAR_MED_RESUMED_DATE_PA", "BLOOD_SUGAR_MEDICATION_PA",
                    "C_PEPTIDE", "C_PEPTIDE_PA_TCR", "C_PEPTIDE_PA_TRR", "C_PEPTIDEDATE", "DAYSWAIT_CHRON_PA",
                    "DGN2_TCR", "DIAG_PA", "END_STAT_PA", "ENTERIC_DRAIN", "ENTERIC_DRAIN_DT", "GSTATUS_PA",
                    "GTIME_PA", "HBA1C_PA_TCR", "HBA1C_PA_TRR", "INFECT_PA", "INSULIN_DOSAGE_OLD_PA",
                    "INSULIN_DOSAGE_PA", "INSULIN_DUR_DON", "INSULIN_DURATION_PA", "INSULIN_PA",
                    "INSULIN_RESUMED_DATE_PA", "LIPASE", "MALIG_TCR_PA", "METHOD_BLOOD_SUGAR_CONTROL_PA", 
                    "NPPAN", "OPER_TECH", "PA_PRESERV_TM", "PANCREATIT_PA", "PREV_PA_TX", "PRI_PAYMENT_CTRY_TCR_PA",
                    "PRI_PAYMENT_CTRY_TRR_PA", "PRI_PAYMENT_TCR_PA", "PRI_PAYMENT_TRR_PA", "PRVTXDIF_PA",
                    "PX_NON_COMPL_PA", "REJ_ACUTE_PA", "REJ_CHRONIC_PA", "REJ_HYPER_PA", "REJCNF_PA", "REJTRT_PA",
                    "RETXDATE_PA", "TRTREJ1Y_PA", "TRTREJ6M_PA", "TX_PROCEDUR_TY_PA", "TX_TYPE", "VASC_MGMT",
                    "VEN_EXT_GRF", 'Kidney_Pancreas_PRA', 'Kidney_Malig_Followup', 'Kidney_Followup', 
                   ]

kidpan_duplicate_columns = ['DGN_TCR', 'AGE_BIN', 'INIT_AGE_BIN', 'INIT_AGE', 'AGE_BIN_DON', 'AGE_BIN_REC']

# other unwanted columns (e.g. PTIME measures time until death while GTIME_KI measures time until kidney failure)
kidpan_other_to_delete = ["PTIME", "PSTATUS", "PX_STAT", "PX_STAT_DATE", "PT_CODE", 'END_OPO_CTR_CODE', 'CTR_CODE',
                         'RECOV_FACILITY_CODE', 'LISTING_CTR_CODE', 'VAL_DT_TCR', 'VAL_DT_TRR', 'VAL_DT_LDR',
                          'Kidney_Pancreas_WL_History', 'Kidney_Pancreas_Immuno_Followup', 'Kidney_Pancreas_HLA',
                          'Kidney_Pancreas_Immuno_Discharge', '	INIT_OPO_CTR_CODE', 
                         ]

kidpan_unwanted_columns = list(set(kidpan_post_transplant_columns + kidpan_at_transplant_columns +
                                   kidpan_irrelevant_columns + kidpan_pancreas_columns + 
                                   kidpan_duplicate_columns + kidpan_other_to_delete))

#### Unwanted columns from the Living_Donor table

In [ ]:
living_other_organ_columns = ["ARRHYTHMIA", "ARRHYTHMIA_POSTOP", "BILIARY_COMP", "BILIARY_COMP_GRADE", "BIOPSY_LI",
                             "INTRAOP_COMP", "INTRAOP_COMP_REASON", "LI_PROC_TY", "LIVER_RECOV", "LU_COMP",
                              "LU_COMP_REASON", "LU_PROC_TY", "LUNG_RECOV", "SACRIFICE_LOBE", "THORAC_TUBES",
                              
                             ]
living_post_transplant_columns = ["BP_POSTOP_DIAST", "BP_POSTOP_SYST", "COD", "DEATH_DT", "FFP_UNITS", 
                                  "HYPERTENSION", "INIT_DISCHARGE_DT", "KI_CREAT_POSTOP", "KIDNEY_RECOV", 
                                  "OTH_COMP_KI", "OTH_COMP_KI_INTER", "OTH_COMP_LI", "OTH_COMP_LI_INTER",
                                  "OTH_INTER_PROC_KI", "OTH_INTER_PROC_KI_DT", "OTH_INTER_PROC_LI",
                                  "OTH_INTER_PROC_LI_DT", "PLATELETS_UNITS", "POSTOP_ALBUM", "POSTOP_ALK_PHOS",
                                  "POSTOP_BILI", "POSTOP_CREAT_LI", "POSTOP_INR", "POSTOP_SGOT_AST",
                                  "POSTOP_SGPT_ALT", "POSTOP_TEST_DT", "POSTOP_URINE_PROTEIN",
                                  "POSTOP_URINE_RATIO", "PRBC_UNITS", "REOP_BILIARY", "REOP_BILIARY_DT", 
                                  "REOP_BLEED_KI", "REOP_BLEED_KI_DT", "REOP_BLEED_LI", "REOP_BLEED_LI_DT",
                                  "REOP_BOWEL_KI", "REOP_BOWEL_KI_DT", "REOP_BOWEL_LI", "REOP_BOWEL_LI_DT", 
                                  "REOP_HERNIA_KI", "REOP_HERNIA_KI_DT", "REOP_HERNIA_LI", "REOP_HERNIA_LI_DT", 
                                  "REOP_LI_FAIL", "REOP_LI_FAIL_DT", "REOP_OTH_KI", "REOP_OTH_KI_DT",
                                  "REOP_OTH_LI", "REOP_OTH_LI_DT", "REOP_VASC_KI", "REOP_VASC_KI_DT", 
                                  "REOP_VASC_LI", "REOP_VASC_LI_DT", "REOPERATION_KI", "REOPERATION_LI",
                                  "VASC_COMP_KI", "VASC_COMP_KI_INTER", "VASC_COMP_LI", "VASC_COMP_LI_INTER",
                                  "WGT_KG"
                                 ]

living_at_transplant = ["CONVERT_OPEN_KI", "CONVERT_OPEN_LU", "KI_PROC_TY", 'ORG_RECOVERY_DT' ]

living_other_to_delete = ["DONOR_ID", "STATUS_LDR",'Living_Donor_Follow',  ]

living_unwanted_columns = list(set(living_other_organ_columns + living_post_transplant_columns +
                                   living_at_transplant + living_other_to_delete))

#### Deleting the columns identified as unwanted

In [ ]:
new_columns = [x for x in reduced_columns if x not in kidpan_unwanted_columns and 
               x not in living_unwanted_columns]
len(new_columns)

In [ ]:
kidpan_living = kidpan_living[new_columns]
kidpan_living

#### Checking the % of NaN in the remaining columns 

In [ ]:
NaN_count_df = pd.DataFrame(columns=["Feature", "Percentage of NaN"])
for col in kidpan_living.columns.to_list():
    totalRows = kidpan_living.shape[0]
    count = kidpan_living[col].isna().sum()
    percentage = count / totalRows * 100
    new_row = {"Feature": col, "Percentage of NaN": percentage}
    NaN_count_df = pd.concat([NaN_count_df, pd.DataFrame([new_row])])
    
NaN_count_df = NaN_count_df.sort_values(by="Percentage of NaN")
NaN_count_df = NaN_count_df.reset_index(drop=True)
NaN_count_df[0:220]

# Categorical and Numerical values

#### Checking if columns are correctly identified as categorical or numerical

In [ ]:
data_type_df = pd.DataFrame({ 'nunique': kidpan_living.nunique(), 'dtype': kidpan_living.dtypes})
data_type_df = data_type_df.sort_values(by='nunique', ascending=True)
data_type_df = data_type_df.reset_index()
data_type_df = data_type_df.rename(columns={'index': 'feature'})
data_type_df

In [ ]:
sorted_columns = data_type_df['feature'].to_list()
kidpan_living.reindex(columns=sorted_columns)

#### No point in keeping columns with 1 unique value

In [ ]:
kidpan_living = kidpan_living.drop(['SHARE_TY', 'PAYBACK', 'REM_CD'], axis=1)

#### Some columns are categorical but have numerical values representing the categories and are therefore incorrectly identified as numerical

In [ ]:
categorical_incorrectly_identified_as_numerical = ['GSTATUS_KI', 'ETHNICITY', 'ABO_MAT',
                                                  'BW6', 'BW4', 'DBW6', 'DIAB', 'DBW4', 'HIST_HYPER',
                                                  'CITIZENSHIP_DON', 'DDR53', 'MARITAL_STAT', 'CITIZENSHIP_REC',
                                                  'DR53_2', 'HAPLO_TY_MATCH_DON', 'EDUCATION_DON', 'ETHCAT',
                                                   'EDUCATION_REC', 'DR53', 'END_STAT_KI', 'ETHCAT_DON', 'DDR51',
                                                   'DR51_2', 'DR51', 'REGION_DON', 'DR52_2', 'DDR52', 'REGION_REC',
                                                   'PRI_PAYMENT_TCR_KI', 'FUNC_STAT', 'LIV_DON_TY', 'FUNC_STAT_TCR',
                                                   'DQ1', 'DQ2', 'HIST_CANCER', 'CANCER_SITE_DON', 'DDQ2', 'DDQ1',
                                                   'C1', 'DC1', 'C2', 'A1', 'A2', 'DC2', 'DA1', 'DA2', 'RA2', 'DDP1',
                                                   'RA1', 'DDR1', 'DR1', 'DR2', 'DDR2', 'RDR1', 'RDR2', 'B2',
                                                   'DB1', 'B1', 'DB2', 'RB2', 'RB1', 'DDP2', 'DIAG_KI']
kidpan_living[categorical_incorrectly_identified_as_numerical] = kidpan_living[categorical_incorrectly_identified_as_numerical].astype('object')

In [ ]:
# possibly drop these to prevent overcomplicanting, they are already included in HLAMIS
HLA_columns = ['BW6', 'BW4', 'DBW6', 'DBW4','DDR53','DR53_2','DR53','DDR51','DR51_2', 'DR51', 'DR52',
                   'DR52_2', 'DDR52','DQ1', 'DQ2', 'DDQ2', 'DDQ1', 'C1', 'DC1', 'C2', 'A1', 'A2', 'DC2',
                   'DA1', 'DA2', 'RA2', 'DDP1', 'RA1', 'DDR1', 'DR1', 'DR2', 'DDR2', 'RDR1', 'RDR2', 'B2',
                  'DB1', 'B1', 'DB2', 'RB2', 'RB1', 'DDP2', 'AMIS', 'BMIS', 'DRMIS' 
                  ]

In [ ]:
kidpan_living = kidpan_living.drop(HLA_columns, axis=1)

#### Date columns are strings - transforming them into numerical by only using year

In [ ]:
date_columns = ['WT_QUAL_DATE', 'END_DATE', 'TX_DATE', 'DON_DATE', 'INIT_DATE', 'DIAL_DATE']
for col in date_columns:
    years = []
    for value in kidpan_living[col]:
        if pd.notna(value):
            date_string = value.split("'")[3]
            year_string = date_string.split("-")[0]
            years.append(year_string)
        else:
            years.append(np.nan)
    kidpan_living[col] = years
    kidpan_living[col] =  pd.to_numeric(kidpan_living[col], errors='coerce').astype('float64')

In [ ]:
kidpan_living[date_columns].dtypes

In [ ]:
kidpan_living

# Training the model

#### Dropping rows with missing target values

In [ ]:
kidpan_living = kidpan_living.dropna(subset=['GSTATUS_KI', 'GTIME_KI'])
kidpan_living.shape

In [ ]:
groups = kidpan_living.groupby(["GSTATUS_KI", "DWFG_KI"]).size()
groups

In [ ]:
kidpan_living.to_csv('csv_data/final_kidpan_living.csv', index=False)

#### Splitting values into target and feature variables

In [ ]:
y = kidpan_living[["GSTATUS_KI", "GTIME_KI"]]
X = kidpan_living.drop(['GSTATUS_KI' ,'GTIME_KI', 'END_DATE', 'DWFG_KI'], axis=1)
del kidpan_living
X.shape

#### Formating y for RSF

In [ ]:
y_struct = np.zeros(y.shape[0], dtype=[('event', bool), ('time', float)])
y_struct['event'] = y.iloc[:, 0] == 1
y_struct['time'] = y.iloc[:, 1]

#### Splitting X into train and test data

In [ ]:
random_state = 10

X_train, X_test, y_train, y_test = train_test_split(
    X, y_struct, test_size=0.2, random_state=random_state)

#### Defining categorical and numerical columns

In [ ]:
all_cols = X.columns.tolist()
categorical_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
numerical_cols = list(filter(lambda x: x not in categorical_cols, all_cols))

#### Imputing numerical columns

In [ ]:
numerical_transformer = SimpleImputer(strategy='mean')

In [ ]:
numerical_imputed_X_train = pd.DataFrame(numerical_transformer.fit_transform(X_train[numerical_cols]))
numerical_imputed_X_test = pd.DataFrame(numerical_transformer.transform(X_test[numerical_cols]))
numerical_imputed_X_train.columns = X_train[numerical_cols].columns
numerical_imputed_X_test.columns = X_test[numerical_cols].columns
numerical_imputed_X_train

#### Imputing categorical columns

In [ ]:
categorical_imputer = SimpleImputer(strategy='most_frequent')

In [ ]:
categorical_imputed_X_train = pd.DataFrame(categorical_imputer.fit_transform(X_train[categorical_cols]))
categorical_imputed_X_test = pd.DataFrame(categorical_imputer.transform(X_test[categorical_cols]))
categorical_imputed_X_train.columns = X_train[categorical_cols].columns
categorical_imputed_X_test.columns = X_test[categorical_cols].columns

In [ ]:
categorical_imputed_X_train = categorical_imputed_X_train.astype(str)
categorical_imputed_X_test = categorical_imputed_X_test.astype(str)

#### One-hot encoding categorical columns

In [ ]:
enc = OneHotEncoder(sparse=False, handle_unknown = 'ignore')
encoded_array_train = enc.fit_transform(categorical_imputed_X_train)
encoded_array_test = enc.transform(categorical_imputed_X_test)
encoded_columns_train=enc.get_feature_names_out(categorical_imputed_X_train.columns)
encoded_columns_test=enc.get_feature_names_out(categorical_imputed_X_test.columns)
categorical_encoded_train = pd.DataFrame(encoded_array_train, columns=encoded_columns_train)
categorical_encoded_test = pd.DataFrame(encoded_array_test, columns=encoded_columns_test)

#### Merging back numerical and categorical columns

In [ ]:
X_train = numerical_imputed_X_train.merge(categorical_encoded_train, left_index=True, right_index=True)
X_test = numerical_imputed_X_test.merge(categorical_encoded_test, left_index=True, right_index=True)
X_train

#### Defining, tranining and scoring the RSF model 

In [ ]:
rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=10,
                           n_jobs=-1,
                           random_state=random_state)

In [ ]:
rsf.fit(X_train, y_train)

In [ ]:
rsf.score(X_test, y_test)

#### Calculating permutation importance

In [ ]:
result = permutation_importance(rsf, X_test, y_test, n_repeats=2, random_state=random_state, 
                                n_jobs=1, max_samples=0.1)

In [ ]:
variable_importance_df = pd.DataFrame(
    {k: result[k] for k in ("importances_mean", "importances_std",)},
    index=X_test.columns
).sort_values(by="importances_mean", ascending=False)

variable_importance_df

In [ ]:
reindexed_variable_importance_df = variable_importance_df.reset_index()
renamed_variable_importance_df = reindexed_variable_importance_df.rename(columns={'index': 'Feature'})
renamed_variable_importance_df

In [ ]:
renamed_variable_importance_df.to_csv('variable_importances/new_imputed_livdon_n2.csv', index=False)